In [0]:
import os

dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]
for npz_file in npz_files:
    npz_file_path = dataset_dir + "/" + npz_file
    print(f"npz_file_path : {npz_file_path}")

In [9]:
import os
import subprocess

import numpy as np
import torch
from sklearn.metrics import roc_auc_score

# 设定路径
# dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
dataset_dir = "datasets/lorenz/F10/time200"  # 存储lorenz数据的文件夹
output_dir = "output/logs"  # 存储输出日志的目录

# 获取所有的npz文件
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]

# 准备AUROC结果存储
auroc_results = []

# 遍历每个文件，执行main.py，计算AUROC
for npz_file in npz_files:
    print(f"Processing file: {npz_file}")

    # 数据文件路径
    npz_file_path = dataset_dir + "/" + npz_file

    # 构建命令行参数
    command = [
        "python", "main2.py",
        "--dataset", "lorenz",
        "--dsid", "1",  # 这里使用1，假设每个数据集有一个dsid
        "--model", "sru",
        "--sruname", "10",  # 设置为10，按照要求
        "--T", "200",  # 设置时间步数
        "--F", "10",  # 设置F
        "--nepochs", "2000",  # 训练2000个epoch
        "--mu1", "0.021544",  # mu1的值
        "--mu2", "0.031623",  # mu2的值
        "--mu3", "0",  # mu3的值
        "--lr", "0.001",  # 学习率
        "--joblog", output_dir + "/" + f"{npz_file}_output.npz",  # 输出路径
        "--inputfile", npz_file_path  # 传递数据文件路径
    ]

    # 执行命令
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {npz_file}: {e}")
        continue

    # 输出文件路径（假设每个文件的结果都保存在对应joblog输出路径）
    output_file_path = output_dir + "/" + f"{npz_file}_output.npz"

    # 读取输出的结果文件（假设它包含了Gest和Gref）
    try:
        result = np.load(output_file_path)
        Gest = torch.tensor(result["Gest"])  # 预测值
        Gref = torch.tensor(result["Gref"])  # 真实值

        # 计算 AUROC
        # 假设Gest和Gref都是每个节点的预测和真实值
        # 如果你有多个节点，计算每个节点的AUROC并取平均
        auroc_values = []
        for i in range(Gest.shape[0]):  # 遍历每个节点
            try:
                auroc = roc_auc_score(Gref[i].cpu().numpy(), Gest[i].cpu().numpy())
                auroc_values.append(auroc)
            except ValueError as ve:
                print(f"Error computing AUROC for node {i}: {ve}")
                auroc_values.append(np.nan)

        # 计算该文件的平均AUROC值
        avg_auroc = np.nanmean(auroc_values)
        auroc_results.append((npz_file, avg_auroc))
        print(f"AUROC for {npz_file}: {avg_auroc:.4f}")

    except Exception as e:
        print(f"Error reading output file for {npz_file}: {e}")
        continue

# 输出所有文件的AUROC结果
print("\nAll AUROC Results:")
for file, auroc in auroc_results:
    print(f"{file}: {auroc:.4f}")


Processing file: lorenz-381-F10-200.npz
Computational Resource: cpu
tensor([[10.7732, 11.7255, 11.1167,  ...,  0.3064,  0.6288,  1.5719],
        [10.9542,  4.5473, -4.8443,  ...,  1.7207,  2.8269,  4.3059],
        [-0.5828, -6.5629, -4.3325,  ...,  7.1175,  6.8424,  6.3382],
        ...,
        [ 0.2805, -0.1437,  1.3464,  ..., 14.6067, 15.3504, 15.0539],
        [-0.4749,  0.2884,  1.6568,  ...,  3.2846,  3.5197, -1.5238],
        [ 1.1973,  1.8182,  3.3854,  ...,  5.0264, -0.1825, -0.5718]])
shape: torch.Size([10, 200])
node = 0
1041
node = 1
1041
node = 2
1041
node = 3
1041
node = 4
1041
node = 5
1041
node = 6
1041
node = 7
1041
node = 8
1041
node = 9
1041
[[1 1 0 0 0 0 0 0 1 1]
 [1 1 1 0 0 0 0 0 0 1]
 [1 1 1 1 0 0 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 0]
 [0 0 1 1 1 1 0 0 0 0]
 [0 0 0 1 1 1 1 0 0 0]
 [0 0 0 0 1 1 1 1 0 0]
 [0 0 0 0 0 1 1 1 1 0]
 [0 0 0 0 0 0 1 1 1 1]
 [1 0 0 0 0 0 0 1 1 1]]
tensor([[0.7725, 0.3963, 0.3713, 0.3541, 0.3599, 0.2867, 0.2922, 0.3430, 0.4005,
         0.5784],

In [8]:
import os
import subprocess

import numpy as np
import torch
from sklearn.metrics import roc_auc_score

# 设定路径
# dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
dataset_dir = "datasets/lorenz/F10/time500"  # 存储lorenz数据的文件夹
output_dir = "output/logs"  # 存储输出日志的目录

# 获取所有的npz文件
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]

# 准备AUROC结果存储
auroc_results = []

# 遍历每个文件，执行main.py，计算AUROC
for npz_file in npz_files:
    print(f"Processing file: {npz_file}")

    # 数据文件路径
    npz_file_path = dataset_dir + "/" + npz_file

    # 构建命令行参数
    command = [
        "python", "main2.py",
        "--dataset", "lorenz",
        "--dsid", "1",  # 这里使用1，假设每个数据集有一个dsid
        "--model", "sru",
        "--sruname", "10",  # 设置为10，按照要求
        "--T", "500",  # 设置时间步数
        "--F", "10",  # 设置F
        "--nepochs", "2000",  # 训练2000个epoch
        "--mu1", "0.021544",  # mu1的值
        "--mu2", "0.031623",  # mu2的值
        "--mu3", "0",  # mu3的值
        "--lr", "0.001",  # 学习率
        "--joblog", output_dir + "/" + f"{npz_file}_output.npz",  # 输出路径
        "--inputfile", npz_file_path  # 传递数据文件路径
    ]

    # 执行命令
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {npz_file}: {e}")
        continue

    # 输出文件路径（假设每个文件的结果都保存在对应joblog输出路径）
    output_file_path = output_dir + "/" + f"{npz_file}_output.npz"

    # 读取输出的结果文件（假设它包含了Gest和Gref）
    try:
        result = np.load(output_file_path)
        Gest = torch.tensor(result["Gest"])  # 预测值
        Gref = torch.tensor(result["Gref"])  # 真实值

        # 计算 AUROC
        # 假设Gest和Gref都是每个节点的预测和真实值
        # 如果你有多个节点，计算每个节点的AUROC并取平均
        auroc_values = []
        for i in range(Gest.shape[0]):  # 遍历每个节点
            try:
                auroc = roc_auc_score(Gref[i].cpu().numpy(), Gest[i].cpu().numpy())
                auroc_values.append(auroc)
            except ValueError as ve:
                print(f"Error computing AUROC for node {i}: {ve}")
                auroc_values.append(np.nan)

        # 计算该文件的平均AUROC值
        avg_auroc = np.nanmean(auroc_values)
        auroc_results.append((npz_file, avg_auroc))
        print(f"AUROC for {npz_file}: {avg_auroc:.4f}")

    except Exception as e:
        print(f"Error reading output file for {npz_file}: {e}")
        continue

# 输出所有文件的AUROC结果
print("\nAll AUROC Results:")
for file, auroc in auroc_results:
    print(f"{file}: {auroc:.4f}")


Processing file: lorenz-169-F10-500.npz
Computational Resource: cpu
tensor([[-0.4719,  2.3044,  3.4747,  ...,  2.6430,  2.8964,  3.4605],
        [ 2.2450,  4.1074,  7.1179,  ..., 11.7417, 10.5587,  8.7491],
        [ 9.9247,  8.9686,  2.9052,  ..., -0.8356, -7.6320, -4.2512],
        ...,
        [ 3.4964,  2.5382,  2.4924,  ...,  4.0678,  2.2567,  0.4528],
        [ 8.8259,  8.6446,  8.5316,  ...,  5.0953,  2.9231,  2.3225],
        [-2.1671, -3.1036, -1.1332,  ..., -1.2870, -0.5012,  0.7894]])
shape: torch.Size([10, 500])
node = 0
1041
node = 1
1041
node = 2
1041
node = 3
1041
node = 4
1041
node = 5
1041
node = 6
1041
node = 7
1041
node = 8
1041
node = 9
1041
[[1 1 0 0 0 0 0 0 1 1]
 [1 1 1 0 0 0 0 0 0 1]
 [1 1 1 1 0 0 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 0]
 [0 0 1 1 1 1 0 0 0 0]
 [0 0 0 1 1 1 1 0 0 0]
 [0 0 0 0 1 1 1 1 0 0]
 [0 0 0 0 0 1 1 1 1 0]
 [0 0 0 0 0 0 1 1 1 1]
 [1 0 0 0 0 0 0 1 1 1]]
tensor([[0.7570, 0.3932, 0.4183, 0.3566, 0.3180, 0.2369, 0.2713, 0.3419, 0.4302,
         0.5967],

In [11]:
import os
import subprocess

import numpy as np
import torch
from sklearn.metrics import roc_auc_score

# 设定路径
# dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
dataset_dir = "datasets/lorenz/F10/time1000"  # 存储lorenz数据的文件夹
output_dir = "output/logs"  # 存储输出日志的目录

# 获取所有的npz文件
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]

# 准备AUROC结果存储
auroc_results = []

# 遍历每个文件，执行main.py，计算AUROC
for npz_file in npz_files:
    print(f"Processing file: {npz_file}")

    # 数据文件路径
    npz_file_path = dataset_dir + "/" + npz_file

    # 构建命令行参数
    command = [
        "python", "main2.py",
        "--dataset", "lorenz",
        "--dsid", "1",  # 这里使用1，假设每个数据集有一个dsid
        "--model", "sru",
        "--sruname", "10",  # 设置为10，按照要求
        "--T", "1000",  # 设置时间步数
        "--F", "10",  # 设置F
        "--nepochs", "2000",  # 训练2000个epoch
        "--mu1", "0.021544",  # mu1的值
        "--mu2", "0.031623",  # mu2的值
        "--mu3", "0",  # mu3的值
        "--lr", "0.001",  # 学习率
        "--joblog", output_dir + "/" + f"{npz_file}_output.npz",  # 输出路径
        "--inputfile", npz_file_path  # 传递数据文件路径
    ]

    # 执行命令
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {npz_file}: {e}")
        continue

    # 输出文件路径（假设每个文件的结果都保存在对应joblog输出路径）
    output_file_path = output_dir + "/" + f"{npz_file}_output.npz"

    # 读取输出的结果文件（假设它包含了Gest和Gref）
    try:
        result = np.load(output_file_path)
        Gest = torch.tensor(result["Gest"])  # 预测值
        Gref = torch.tensor(result["Gref"])  # 真实值

        # 计算 AUROC
        # 假设Gest和Gref都是每个节点的预测和真实值
        # 如果你有多个节点，计算每个节点的AUROC并取平均
        auroc_values = []
        for i in range(Gest.shape[0]):  # 遍历每个节点
            try:
                auroc = roc_auc_score(Gref[i].cpu().numpy(), Gest[i].cpu().numpy())
                auroc_values.append(auroc)
            except ValueError as ve:
                print(f"Error computing AUROC for node {i}: {ve}")
                auroc_values.append(np.nan)

        # 计算该文件的平均AUROC值
        avg_auroc = np.nanmean(auroc_values)
        auroc_results.append((npz_file, avg_auroc))
        print(f"AUROC for {npz_file}: {avg_auroc:.4f}")

    except Exception as e:
        print(f"Error reading output file for {npz_file}: {e}")
        continue

# 输出所有文件的AUROC结果
print("\nAll AUROC Results:")
for file, auroc in auroc_results:
    print(f"{file}: {auroc:.4f}")


Processing file: lorenz-169-F10-1000.npz
Computational Resource: cpu
tensor([[ 2.2037,  3.6098, -0.2965,  ..., -2.3251, -1.2428, -0.5167],
        [-1.8406,  0.0763,  1.1999,  ..., -6.6357, -5.0896, -3.5993],
        [ 1.8289,  2.7404,  3.7013,  ...,  2.2962, -2.3941, -3.9358],
        ...,
        [ 3.5459,  0.8456, -0.6586,  ...,  1.8071, -0.6074, -2.8640],
        [13.8677, 14.0580, 13.4736,  ...,  0.8452,  0.9258,  3.6444],
        [-0.8433,  1.0665,  5.0794,  ..., -0.4879,  0.1999,  1.3539]])
shape: torch.Size([10, 1000])
node = 0
1041
node = 1
1041
node = 2
1041
node = 3
1041
node = 4
1041
node = 5
1041
node = 6
1041
node = 7
1041
node = 8
1041
node = 9
1041
[[1 1 0 0 0 0 0 0 1 1]
 [1 1 1 0 0 0 0 0 0 1]
 [1 1 1 1 0 0 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 0]
 [0 0 1 1 1 1 0 0 0 0]
 [0 0 0 1 1 1 1 0 0 0]
 [0 0 0 0 1 1 1 1 0 0]
 [0 0 0 0 0 1 1 1 1 0]
 [0 0 0 0 0 0 1 1 1 1]
 [1 0 0 0 0 0 0 1 1 1]]
tensor([[0.7629, 0.4283, 0.3406, 0.3791, 0.3136, 0.2194, 0.2582, 0.3352, 0.5389,
         0.5680

In [ ]:
import os
import subprocess

import numpy as np
import torch
from sklearn.metrics import roc_auc_score

# 设定路径
# dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
dataset_dir = "datasets/lorenz/F20/time1000"  # 存储lorenz数据的文件夹
output_dir = "output/logs"  # 存储输出日志的目录

# 获取所有的npz文件
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]

# 准备AUROC结果存储
auroc_results = []

# 遍历每个文件，执行main.py，计算AUROC
for npz_file in npz_files:
    print(f"Processing file: {npz_file}")

    # 数据文件路径
    npz_file_path = dataset_dir + "/" + npz_file

    # 构建命令行参数
    command = [
        "python", "main2.py",
        "--dataset", "lorenz",
        "--dsid", "1",  # 这里使用1，假设每个数据集有一个dsid
        "--model", "sru",
        "--sruname", "10",  # 设置为10，按照要求
        "--T", "1000",  # 设置时间步数
        "--F", "10",  # 设置F
        "--nepochs", "2000",  # 训练2000个epoch
        "--mu1", "0.021544",  # mu1的值
        "--mu2", "0.031623",  # mu2的值
        "--mu3", "0",  # mu3的值
        "--lr", "0.001",  # 学习率
        "--joblog", output_dir + "/" + f"{npz_file}_output.npz",  # 输出路径
        "--inputfile", npz_file_path  # 传递数据文件路径
    ]

    # 执行命令
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {npz_file}: {e}")
        continue

    # 输出文件路径（假设每个文件的结果都保存在对应joblog输出路径）
    output_file_path = output_dir + "/" + f"{npz_file}_output.npz"

    # 读取输出的结果文件（假设它包含了Gest和Gref）
    try:
        result = np.load(output_file_path)
        Gest = torch.tensor(result["Gest"])  # 预测值
        Gref = torch.tensor(result["Gref"])  # 真实值

        # 计算 AUROC
        # 假设Gest和Gref都是每个节点的预测和真实值
        # 如果你有多个节点，计算每个节点的AUROC并取平均
        auroc_values = []
        for i in range(Gest.shape[0]):  # 遍历每个节点
            try:
                auroc = roc_auc_score(Gref[i].cpu().numpy(), Gest[i].cpu().numpy())
                auroc_values.append(auroc)
            except ValueError as ve:
                print(f"Error computing AUROC for node {i}: {ve}")
                auroc_values.append(np.nan)

        # 计算该文件的平均AUROC值
        avg_auroc = np.nanmean(auroc_values)
        auroc_results.append((npz_file, avg_auroc))
        print(f"AUROC for {npz_file}: {avg_auroc:.4f}")

    except Exception as e:
        print(f"Error reading output file for {npz_file}: {e}")
        continue

# 输出所有文件的AUROC结果
print("\nAll AUROC Results:")
for file, auroc in auroc_results:
    print(f"{file}: {auroc:.4f}")


Processing file: lorenz-381-F20-1000.npz
Computational Resource: cpu
tensor([[  7.9987,   3.9903,   3.6880,  ...,   0.9617,  -8.6353,  -5.2162],
        [-10.2784, -10.2385,  -7.4614,  ...,  26.7369,  24.4717,   0.9279],
        [  2.9866,   5.6446,  -4.7878,  ...,   2.4040,   0.8049,  14.4030],
        ...,
        [  3.2711,  16.1524,  14.1159,  ...,   1.7868,  -0.8768,   0.1724],
        [-15.8695,   7.0099,  13.2193,  ...,  11.0392,  11.6731,  13.1591],
        [  8.9278,   9.2394,  -7.5752,  ...,  -8.0850,  -9.9189, -18.4429]])
shape: torch.Size([10, 1000])
node = 0
1041
node = 1
1041
node = 2
1041
node = 3
1041
node = 4
1041
node = 5
1041
node = 6
1041
node = 7
1041
node = 8
1041
node = 9
1041
[[1 1 0 0 0 0 0 0 1 1]
 [1 1 1 0 0 0 0 0 0 1]
 [1 1 1 1 0 0 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 0]
 [0 0 1 1 1 1 0 0 0 0]
 [0 0 0 1 1 1 1 0 0 0]
 [0 0 0 0 1 1 1 1 0 0]
 [0 0 0 0 0 1 1 1 1 0]
 [0 0 0 0 0 0 1 1 1 1]
 [1 0 0 0 0 0 0 1 1 1]]
tensor([[0.6554, 0.4240, 0.4291, 0.2981, 0.2453, 0.3421, 0.2

In [10]:
import torch

print(torch.cuda.is_available())  # 检查是否能够使用 GPU
print(torch.version.cuda)  # 查看当前 CUDA 版本
print(torch.cuda.current_device())  # 查看当前设备的 ID


True
10.0.130
0


In [6]:
torch.cuda.empty_cache()
